In [3]:
import csv
import json
import os
import requests
from datetime import datetime

import slugify
import pandas as pd

# Read Movielens dataset

In [4]:
df = pd.read_csv("../data/ml-25m/links.csv", sep=",", encoding="latin", dtype="str")

In [5]:
df = df.sample(100)

# Enrich with OMDB

In [6]:
with open("../omdb.key", "r") as f:
    OMDB_KEY = f.read().strip()

In [7]:
def get_movie_from_omdb(imdb_id):
    movie = requests.get(
        url="http://www.omdbapi.com/",
        params={"apikey": OMDB_KEY, "i": "tt{0}".format(imdb_id)}
    ).json()
    
    return movie

In [8]:
movies = list(map(get_movie_from_omdb, df["imdbId"]))

In [9]:
movies_df = pd.DataFrame(movies)
movies_df.columns = list(map(str.lower, movies_df.columns))

In [12]:
def make_slug(title, year):
    year = str(year)[:4]
    timestamp = datetime.strftime(
        datetime.fromisoformat("{0}-01-01".format(year)),
        "%Y/%m/%d"
    )
    slug = slugify.slugify(title)
    
    return "{0}/{1}".format(timestamp, slug)

In [13]:
movies_df["url"] = movies_df.apply(lambda r: make_slug(r["title"], r["year"]), axis=1)

In [15]:
movies_df.to_csv("../data/movies.csv", index=False)